<a href="https://colab.research.google.com/github/Stock-XAI/LLM_server/blob/main/API_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip
!pip install fastapi uvicorn nest-asyncio pyngrok
!pip install transformers accelerate torch pandas yfinance finance-datareader bitsandbytes peft shap matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 135.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 146.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 M

In [2]:
import nest_asyncio
from fastapi import FastAPI, Query
from pyngrok import ngrok
from threading import Thread
import uvicorn

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

import pandas as pd
import yfinance as yf
import FinanceDataReader as fdr
from datetime import datetime, timedelta
import numpy as np
import re
import shap
import matplotlib.pyplot as plt

from huggingface_hub import login
from google.colab import userdata
login(token=userdata.get('HF_TOKEN'))

### 모델 로드 및 준비

In [13]:
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    "capston-team-5/finma-7b-4bit-quantized",
    quantization_config=bnb_cfg,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, "capston-team-5/finma-7b-lora-regression-v2")
tokenizer = AutoTokenizer.from_pretrained("capston-team-5/finma-7b-lora-regression-v2")
model.eval()

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


adapter_config.json:   0%|          | 0.00/873 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/320M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default.we

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
         

### 프롬프트 생성 및 추론 함수

In [22]:
def get_company_name(ticker: str) -> str:
    try:
        if ticker.endswith((".KS", ".KQ")):
            code = ticker.split(".")[0]  # "005930.KS" → "005930"
            stock_info = fdr.StockListing("KRX")
            name = stock_info.loc[stock_info['Code'] == code, 'Name']
            if not name.empty:
                return name.values[0]
        else:
            info = yf.Ticker(ticker).info
            return info.get("longName", ticker)
    except Exception:
        return ticker  # fallback

def generate_prompt(ticker, interval, context, date_str):
    company_name = get_company_name(ticker)
    return (
        f"Using the context below, estimate the rate of change in the closing price of {company_name} on {date_str}.\n"
        "Return the expected value of change as a decimal.\n\n"
        "Context: date, open, high, low, close, volume, change.\n"
        f"{context}\n\nAnswer:"
    )

def generate_response(prompt, max_new_tokens=16):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

def get_prompt(ticker: str, horizon_days: int):
    assert horizon_days in [1, 7, 30], "Only 1, 7, or 30 day horizon supported"
    today = datetime.today()
    predict_date = today + timedelta(days=horizon_days)
    interval, fetch_days = ("1d", 20) if horizon_days == 1 else ("1wk", 70) if horizon_days == 7 else ("1mo", 300)
    start = (today - timedelta(days=fetch_days)).strftime("%Y-%m-%d")
    end = (today + timedelta(days=1)).strftime("%Y-%m-%d")

    data = yf.download(ticker, start=start, end=end, interval=interval)[["Open", "High", "Low", "Close", "Volume"]]
    data = data.reset_index()
    data["Date"] = data["Date"].dt.strftime("%Y-%m-%d")
    data["Change"] = data["Close"].pct_change().fillna(0)

    last_10 = data.tail(10)

    def extract(val):
        return val.iloc[0] if isinstance(val, pd.Series) else val

    is_krx = ticker.endswith((".KS", ".KQ"))
    if is_krx:
        # 한국 종목은 정수로 표기
        context = "\n".join([
            f"{extract(row['Date'])}, {int(round(extract(row['Open'])))}"
            f", {int(round(extract(row['High'])))}"
            f", {int(round(extract(row['Low'])))}"
            f", {int(round(extract(row['Close'])))}"
            f", {int(extract(row['Volume']))}"
            f", {float(extract(row['Change'])):.6f}"
            for _, row in last_10.iterrows()
        ])
    else:
        # 해외 종목은 소수 포함
        context = "\n".join([
            f"{extract(row['Date'])}, {float(extract(row['Open']))}"
            f", {float(extract(row['High']))}"
            f", {float(extract(row['Low']))}"
            f", {float(extract(row['Close']))}"
            f", {int(extract(row['Volume']))}"
            f", {float(extract(row['Change'])):.6f}"
            for _, row in last_10.iterrows()
        ])

    prompt = generate_prompt(ticker, interval, context, predict_date.strftime("%Y-%m-%d"))
    response = generate_response(prompt)

    return prompt

def extract_float(answer_text):
    try:
        parts = re.split(r'[,\n]+', answer_text.strip())
        return float(parts[0])
    except (ValueError, IndexError):
        return 1.0

def run_prediction(ticker: str, horizon_days: int):
    prompt = get_prompt(ticker, horizon_days)
    response = generate_response(prompt)
    answer = extract_float(response.split("Answer:")[-1].strip())

    if abs(answer) > 0.3:
        print("\nError: change value is very large:", answer)

        today = datetime.today()
        interval, fetch_days = ("1d", 15) if horizon_days == 1 else ("1wk", 70) if horizon_days == 7 else ("1mo", 300)
        start = (today - timedelta(days=fetch_days)).strftime("%Y-%m-%d")
        end = (today + timedelta(days=1)).strftime("%Y-%m-%d")

        data = yf.download(ticker, start=start, end=end, interval=interval)[["Close"]]
        data["Change"] = data["Close"].pct_change().fillna(0)
        fallback_value = data["Change"].tail(10).mean()

        return round(fallback_value, 6), prompt, response  # 이상치로 간주

    return answer, prompt, response

In [23]:
import time

start_time = time.time()
result, prompt, response = run_prediction("AAPL", 30)
end_time = time.time()

print("Result:", result)
print("Response:", response)
print(f"Execution time: {end_time - start_time:.4f} seconds")


[*********************100%***********************]  1 of 1 completed


Result: -0.064834
Response: Using the context below, estimate the rate of change in the closing price of Apple Inc. on 2025-07-11.
Return the expected value of change as a decimal.

Context: date, open, high, low, close, volume, change.
2024-09-01, 227.7494435444956, 232.2735342502758, 213.17068429434045, 232.18385314941406, 1232140300, 0.000000
2024-10-01, 228.7160383097679, 236.65812209589106, 220.55472392637267, 225.11868286132812, 930736000, -0.030429
2024-11-01, 220.1959783617407, 236.97698686400494, 218.94039741577308, 236.49867248535156, 891640600, 0.050551
2024-12-01, 236.6990188491845, 259.4740807467281, 236.58928295305697, 249.81736755371094, 977916100, 0.056316
2025-01-01, 248.3309607168036, 248.50056501956442, 218.85208282459496, 235.4320831298828, 1200291700, -0.057583
2025-02-01, 229.43653690110065, 249.39837756115028, 225.1568522177927, 241.2580108642578, 862272300, 0.024746
2025-03-01, 241.47333710051538, 243.71040900827506, 208.1470444344685, 221.83909606933594, 111523

### SHAP/XAI 설명 함수

In [24]:
def explain_prediction(prompt, tokenizer, model):
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)

    inputs_embeds = model.base_model.model.model.embed_tokens(input_ids)
    inputs_embeds.requires_grad_()

    outputs = model(inputs_embeds=inputs_embeds, labels=input_ids)
    loss = outputs.loss
    loss.backward()

    grads = inputs_embeds.grad.abs().sum(dim=-1).squeeze().detach().cpu().numpy()
    grads /= grads.sum()

    tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze().tolist())
    tokens = [t.replace('▁', '▁') for t in tokens]

    return grads, tokens

grads, tokens = explain_prediction(prompt, tokenizer, model)

In [25]:
def group_tokens_to_words(tokens, scores, group_size=7):
    if len(tokens) != len(scores):
        print("[Warning] tokens and scores length mismatch!")

    words, word_indices = [], []
    current_word, current_indices = "", []

    for i, tok in enumerate(tokens):
        if tok in ["<0x0A>", "\\n", "\n"] or re.fullmatch(r"<0x0A>", tok):
            if current_word:
                words.append(current_word)
                word_indices.append(current_indices)
                current_word, current_indices = "", []
            continue
        elif tok.startswith('▁'):
            if current_word:
                words.append(current_word)
                word_indices.append(current_indices)
            current_word = tok[1:]
            current_indices = [i]
        else:
            current_word += tok
            current_indices.append(i)
    if current_word:
        words.append(current_word)
        word_indices.append(current_indices)

    if word_indices:
        max_idx = max(max(idxs) for idxs in word_indices if idxs)

    words = [re.sub(r'<0x[A-Fa-f0-9]+>', '', w) for w in words]
    words = [re.sub(r'\\n', '', w) for w in words]
    words = [w.strip() for w in words if w.strip() != '']

    # 단어별 importance 집계 (IndexError 안전 체크)
    word_scores = []
    error_count = 0
    for indices in word_indices:
        for idx in indices:
            if idx >= len(scores):
                print(f"[IndexError] idx={idx} out of range for scores (len={len(scores)})")
                error_count += 1
        safe_indices = [i for i in indices if i < len(scores)]
        score = sum(scores[i] for i in safe_indices)
        word_scores.append(score)
    if error_count:
        print(f"[Warning] 총 {error_count}건의 out of range 인덱스가 발견되었습니다.")

    print(words)
    words = words[len(words)-1-group_size*10:len(words)-1]
    word_scores = word_scores[len(word_scores)-1-group_size*10:len(word_scores)-1]
    word_scores = [float(s) for s in word_scores]

    print(f"[Debug] 최종 words 길이: {len(words)}")
    print(f"[Debug] 최종 word_scores 길이: {len(word_scores)}")
    print(f"[Debug] words 샘플: {words[:10]} ...")
    return words, word_scores


In [26]:
ticker = 'AAPL'
horizon_days = 7
prompt = get_prompt(ticker, horizon_days)
grads, tokens = explain_prediction(prompt, tokenizer, model)
token_list, token_score_list = group_tokens_to_words(tokens, grads)

print(f"len(tokens) = {len(token_list)}")
print(f"len(grads) = {len(token_score_list)}")
print(token_list)
print(token_score_list)

[*********************100%***********************]  1 of 1 completed


['<s>', 'Using', 'the', 'context', 'below,', 'estimate', 'the', 'rate', 'of', 'change', 'in', 'the', 'closing', 'price', 'of', 'Apple', 'Inc.', 'on', '2025-06-18.', 'Return', 'the', 'expected', 'value', 'of', 'change', 'as', 'a', 'decimal.', 'Context:', 'date,', 'open,', 'high,', 'low,', 'close,', 'volume,', 'change.', '2025-04-07,', '176.96792778953252,', '200.347272648319,', '168.98840160901602,', '197.89048767089844,', '675037600,', '0.051863', '2025-04-14,', '211.16310052244776,', '212.6611361217968,', '192.11806728809685,', '196.72203063964844,', '263763500,', '-0.005905', '2025-04-21,', '193.0168929962579,', '209.47530610539675,', '189.56141759452223,', '209.00592041015625,', '238181400,', '0.062443', '2025-04-28,', '209.7249739890969,', '214.2789995575084,', '201.89524528415058,', '205.08106994628906,', '286233500,', '-0.018779', '2025-05-05,', '202.8340267238252,', '203.83271712573708,', '192.9969201694707,', '198.27000427246094,', '275704500,', '-0.033212', '2025-05-12,', '210

### FastAPI 서버/엔드포인트 구현

In [27]:
nest_asyncio.apply()
app = FastAPI()

@app.get("/predict")
def predict(ticker: str = Query(...), horizon_days: int = Query(...)):
    result, prompt, _ = run_prediction(ticker, horizon_days)
    return {
        "ticker": ticker,
        "horizon_days": horizon_days,
        "prediction_date": (datetime.today() + timedelta(days=horizon_days)).strftime("%Y-%m-%d"),
        "prediction_result": result,
        "prompt": prompt
    }

@app.get("/explain")
def explain(ticker: str = Query(...), horizon_days: int = Query(...)):
    prompt = get_prompt(ticker, horizon_days)
    grads, tokens = explain_prediction(prompt, tokenizer, model)
    token_list, token_score_list = group_tokens_to_words(tokens, grads)

    return {
        "token_list": token_list,
        "token_score_list": token_score_list
    }


### 서버 실행 & ngrok 공개

In [28]:
ngrok_token = userdata.get('NGROK_TOKEN')
!ngrok config add-authtoken $ngrok_token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [29]:
port = 7861
Thread(target=lambda: uvicorn.run(app, host="0.0.0.0", port=port)).start()
public_url = ngrok.connect(port)
print(f"🚀 API Ready: {public_url}/predict?ticker=005930.KS&horizon_days=1")


INFO:     Started server process [714]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 7861): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


🚀 API Ready: NgrokTunnel: "https://dc5f-34-125-217-198.ngrok-free.app" -> "http://localhost:7861"/predict?ticker=005930.KS&horizon_days=1


In [30]:
import time
_time = 0
while(True):
    time.sleep(300)
    _time += 300
    print("Running Time:", _time, "sec")
    continue

Running Time: 300 sec
Running Time: 600 sec
Running Time: 900 sec
Running Time: 1200 sec
Running Time: 1500 sec
Running Time: 1800 sec
Running Time: 2100 sec
Running Time: 2400 sec
Running Time: 2700 sec
Running Time: 3000 sec


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-45' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 163, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1536, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 7861): address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-29-3133922345>", line 2, in <lambd

['<s>', 'Using', 'the', 'context', 'below,', 'estimate', 'the', 'rate', 'of', 'change', 'in', 'the', 'closing', 'price', 'of', 'International', 'Flavors', '&', 'Fragrances', 'Inc.', 'on', '2025-06-12.', 'Return', 'the', 'expected', 'value', 'of', 'change', 'as', 'a', 'decimal.', 'Context:', 'date,', 'open,', 'high,', 'low,', 'close,', 'volume,', 'change.', '2025-05-28,', '76.91999816894531,', '77.11000061035156,', '75.0,', '75.13999938964844,', '1308800,', '-0.027188', '2025-05-29,', '75.69999694824219,', '76.08999633789062,', '75.19000244140625,', '76.08000183105469,', '1026600,', '0.012510', '2025-05-30,', '75.76000213623047,', '76.73999786376953,', '75.1500015258789,', '76.55999755859375,', '2754600,', '0.006309', '2025-06-02,', '76.11000061035156,', '76.51000213623047,', '75.26000213623047,', '75.70999908447266,', '1557400,', '-0.011102', '2025-06-03,', '75.47000122070312,', '76.72000122070312,', '75.11000061035156,', '76.66000366210938,', '1159500,', '0.012548', '2025-06-04,', '76

[*********************100%***********************]  1 of 1 completed



['<s>', 'Using', 'the', 'context', 'below,', 'estimate', 'the', 'rate', 'of', 'change', 'in', 'the', 'closing', 'price', 'of', 'Quest', 'Diagnostics', 'Incorporated', 'on', '2025-06-12.', 'Return', 'the', 'expected', 'value', 'of', 'change', 'as', 'a', 'decimal.', 'Context:', 'date,', 'open,', 'high,', 'low,', 'close,', 'volume,', 'change.', '2025-05-28,', '172.91000366210938,', '173.6699981689453,', '171.0,', '171.3800048828125,', '746500,', '-0.013243', '2025-05-29,', '171.25999450683594,', '172.5800018310547,', '170.58999633789062,', '172.0,', '783200,', '0.003618', '2025-05-30,', '172.22000122070312,', '173.4199981689453,', '171.36000061035156,', '173.33999633789062,', '1612700,', '0.007791', '2025-06-02,', '172.50999450683594,', '175.2100067138672,', '171.0500030517578,', '174.39999389648438,', '876200,', '0.006115', '2025-06-03,', '173.17999267578125,', '174.2100067138672,', '172.02000427246094,', '173.58999633789062,', '769300,', '-0.004644', '2025-06-04,', '173.82000732421875,'

...

[*********************100%***********************]  1 of 1 completed



['<s>', 'Using', 'the', 'context', 'below,', 'estimate', 'the', 'rate', 'of', 'change', 'in', 'the', 'closing', 'price', 'of', 'Jacobs', 'Solutions', 'Inc.', 'on', '2025-06-18.', 'Return', 'the', 'expected', 'value', 'of', 'change', 'as', 'a', 'decimal.', 'Context:', 'date,', 'open,', 'high,', 'low,', 'close,', 'volume,', 'change.', '2025-04-07,', '108.19163600068812,', '117.44545430838366,', '104.91279878502434,', '115.09496307373047,', '6705087,', '0.045671', '2025-04-14,', '117.1294179299163,', '117.50470445618194,', '114.44314446273857,', '115.76652526855469,', '2314314,', '0.005835', '2025-04-21,', '115.54925990915454,', '121.38598003587852,', '112.85310838043452,', '120.0625991821289,', '3373703,', '0.037110', '2025-04-28,', '120.3687434158574,', '125.90918530460972,', '119.40090081712198,', '125.1092300415039,', '5533386,', '0.042033', '2025-05-05,', '124.56605055079869,', '125.26724901646375,', '115.90479596275092,', '121.23783874511719,', '5901632,', '-0.030944', '2025-05-12,'

[*********************100%***********************]  1 of 1 completed



INFO:     43.200.176.190:0 - "GET /predict?ticker=J&horizon_days=7 HTTP/1.1" 200 OK
['<s>', 'Using', 'the', 'context', 'below,', 'estimate', 'the', 'rate', 'of', 'change', 'in', 'the', 'closing', 'price', 'of', 'JPMorgan', 'Chase', '&', 'Co.', 'on', '2025-06-18.', 'Return', 'the', 'expected', 'value', 'of', 'change', 'as', 'a', 'decimal.', 'Context:', 'date,', 'open,', 'high,', 'low,', 'close,', 'volume,', 'change.', '2025-04-07,', '205.77000427246094,', '238.5800018310547,', '202.16000366210938,', '236.1999969482422,', '105603400,', '0.130183', '2025-04-14,', '237.10000610351562,', '239.77999877929688,', '227.92999267578125,', '231.9600067138672,', '42806800,', '-0.017951', '2025-04-21,', '230.6300048828125,', '246.7899932861328,', '226.33999633789062,', '243.5500030517578,', '51188200,', '0.049965', '2025-04-28,', '244.55999755859375,', '253.6199951171875,', '238.42999267578125,', '252.50999450683594,', '43457400,', '0.036789', '2025-05-05,', '251.17999267578125,', '255.8800048828125

KeyboardInterrupt: 